In [20]:
from pathlib import Path
import numpy as np
import re
# import etree
import xml.etree.ElementTree as etree

In [21]:
sdf_path = Path("/Users/stimpfli/Desktop/nmf2-paper/grooming/data/neuromechfly_frontleg_cylinder_antenna_normal.sdf")

body_pos = {}
visual_offsets = {}

link_name = None
visual_name = None
# parse line by line
with open(sdf_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        if link_name and not ("roll" in link_name or "yaw" in link_name):
            if "<pose>" in line:
                pose_start = line.index("<pose>") + 6
                pose = np.array(line[pose_start:-8].split(" ")).astype(float)[:3]
                if not link_name in body_pos:
                    body_pos[link_name] = pose
                else:
                    print("Link already in body_pos", link_name, pose)
                link_name = None
            else:
                pref_line_had_link = False
                print("No pose found", line)

        if "<link name" in line:
            link_name_start = line.index("=") + 1
            link_name = line[link_name_start+1:-3]

        if visual_name:
            visual_name = visual_name.replace("_visual", "")
            if "<pose>" in line:
                pose_start = line.index("<pose>") + 6
                pose = np.array(line[pose_start:-8].split(" ")).astype(float)[:3]
                if not visual_name in visual_offsets:
                    visual_offsets[visual_name] = pose
                else:
                    print("Visual already in visual_offsets", visual_name, pose)
                visual_name = None
            else:
                pref_line_had_visual = False
                visual_name = None
                print("No pose found", line)

        if "<visual name" in line:
            visual_name_start = line.index("=") + 1
            visual_name = line[visual_name_start+1:-3]

Link already in body_pos LFTarsus2 [0. 0. 0.]
Link already in body_pos LFTarsus2 [0. 0. 0.]
Link already in body_pos LFTarsus3 [0. 0. 0.]
Link already in body_pos LFTarsus3 [0. 0. 0.]
Link already in body_pos LFTarsus4 [0. 0. 0.]
Link already in body_pos LFTarsus4 [0. 0. 0.]
Link already in body_pos LFTarsus5 [0. 0. 0.]
Link already in body_pos LFTarsus5 [0. 0. 0.]
Link already in body_pos RFTarsus2 [0. 0. 0.]
Link already in body_pos RFTarsus2 [0. 0. 0.]
Link already in body_pos RFTarsus3 [0. 0. 0.]
Link already in body_pos RFTarsus3 [0. 0. 0.]
Link already in body_pos RFTarsus4 [0. 0. 0.]
Link already in body_pos RFTarsus4 [0. 0. 0.]
Link already in body_pos RFTarsus5 [0. 0. 0.]
Link already in body_pos RFTarsus5 [0. 0. 0.]


In [22]:
abd_chain = ["Thorax", "A1A2", "A3", "A4", "A5", "A6"]

leg_chain = ["Thorax", "Coxa", "Femur", "Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
antenna_chain = ["Thorax", "Head", "Antenna"]
haltere_chain = ["Thorax", "Haltere"]
wing_chain = ["Thorax", "Wing"]
probuscus_chain = ["Thorax", "Head", "Rostrum", "Haustellum"]
eye_chain = ["Thorax", "Head", "Eye"]

centered_body_parts = ["Thorax", "Head", "Haustellum", "Rostrum"]

chain_body_pos = {}

chain_body_pos["Thorax"] = body_pos["Thorax"]

for i in range(len(abd_chain)-1):
    chain_body_pos[abd_chain[i+1]] = body_pos[abd_chain[i+1]] - body_pos[abd_chain[i]]

for side in ["L", "R"]:
    for chain in [antenna_chain, haltere_chain, wing_chain, eye_chain]:
        for i in range(len(chain)-1):
            el_name = chain[i]
            next_el_name = chain[i+1]
            if next_el_name in chain_body_pos:
                continue
            else:
                if not el_name in centered_body_parts:
                    el_name = side + el_name
                if not next_el_name in centered_body_parts:
                    next_el_name = side + next_el_name
                chain_body_pos[next_el_name] = body_pos[next_el_name] - body_pos[el_name]

for side in "LR":
    for seg in "FMH":
        for chain in [leg_chain]:
            for i in range(len(chain)-1):
                el_name = chain[i]
                next_el_name = chain[i+1]
                if next_el_name in chain_body_pos:
                    continue
                else:
                    if not el_name in centered_body_parts:
                        el_name = side + seg + el_name
                    if not next_el_name in centered_body_parts:
                        next_el_name = side + seg + next_el_name
                    chain_body_pos[next_el_name] = body_pos[next_el_name] - body_pos[el_name]

In [23]:
xml_path = Path("/Users/stimpfli/Desktop/flygym_other/flygym/data/mjcf/neuromechfly_deepfly3d_kinorder_ryp.xml")
# parse xml
xml = etree.parse(xml_path)
for key in chain_body_pos:
    print(key, chain_body_pos[key])
    body = xml.find(".//body[@name='{}']".format(key))
    if body is not None:
        final_body_pos = chain_body_pos[key]
        """if key in visual_offsets and not np.all(visual_offsets[key] == 0.0):
            print("Dealing with non zeros visual offset for {}".format(key))
            final_body_pos += visual_offsets[key]
            joints = body.findall("joint")
            assert len(joints) > 0, "No joints found for {}".format(key)
            for joint in body.findall("joint"):
                # offset the joint in the opposite direction as the joint in the sdf is located at the same place as the chain body pos
                joint_pos = np.array([float(x) for x in joint.get("pos").split(" ")])
                joint_pos -= visual_offsets[key]
                joint.set("pos", " ".join([str(x) for x in joint_pos]))"""

        body.set("pos", " ".join([str(x) for x in final_body_pos]))
    else:
        print("Body not found", key)

# save the xml
new_xml_path = xml_path.parent / (xml_path.stem + "_orig.xml")
xml.write(str(new_xml_path))

Thorax [ 4.80766728e-01 -5.75045123e-07  1.21077104e+00]
A1A2 [-7.90423132e-01 -3.45504736e-08 -3.00191750e-01]
A3 [-3.56473582e-01 -1.29882771e-08 -7.06699211e-04]
A4 [-2.37584929e-01 -1.02345910e-09  3.58442776e-04]
A5 [-2.14296277e-01 -2.17298401e-09 -9.41217877e-05]
A6 [-2.68049072e-01  3.24007488e-09  5.03729098e-04]
Head [ 1.71560387e-02 -7.87051535e-08  1.16415322e-07]
LAntenna [0.43270254 0.0669007  0.1395127 ]
Body not found LAntenna
LHaltere [-0.73973447  0.21355629 -0.27376058]
LWing [-0.57237221  0.3698329   0.11341099]
LEye [2.32068007e-01 9.00982622e-08 8.92099924e-02]
RAntenna [ 0.43268263 -0.06800362  0.13904122]
Body not found RAntenna
RHaltere [-0.73973436 -0.21355668 -0.27376082]
RWing [-0.57237231 -0.36983303  0.11341053]
REye [2.32068065e-01 9.02649622e-08 8.92098760e-02]
LFCoxa [-0.06276678  0.27000001 -0.13077341]
LFFemur [ 0.          0.         -0.42548124]
LFTibia [ 0.          0.         -0.70523994]
LFTarsus1 [ 0.          0.         -0.51838206]
LFTarsus2 [

In [24]:
visual_offsets

{'A1A2': array([0., 0., 0.]),
 'A3': array([0., 0., 0.]),
 'A4': array([0., 0., 0.]),
 'A5': array([0., 0., 0.]),
 'A6': array([0., 0., 0.]),
 'Haustellum': array([0., 0., 0.]),
 'Head': array([0., 0., 0.]),
 'LAntenna': array([0., 0., 0.]),
 'LEye': array([0., 0., 0.]),
 'LFCoxa': array([ 0.    ,  0.    , -0.2125]),
 'LFFemur': array([ 0.   ,  0.   , -0.353]),
 'LFTarsus1': array([ 0.    ,  0.    , -0.3315]),
 'LFTibia': array([ 0.   ,  0.   , -0.259]),
 'LHCoxa': array([0., 0., 0.]),
 'LHFemur': array([0., 0., 0.]),
 'LHTarsus1': array([0., 0., 0.]),
 'LHTarsus2': array([0., 0., 0.]),
 'LHTarsus3': array([0., 0., 0.]),
 'LHTarsus4': array([0., 0., 0.]),
 'LHTarsus5': array([0., 0., 0.]),
 'LHTibia': array([0., 0., 0.]),
 'LHaltere': array([0., 0., 0.]),
 'LMCoxa': array([0., 0., 0.]),
 'LMFemur': array([0., 0., 0.]),
 'LMTarsus1': array([0., 0., 0.]),
 'LMTarsus2': array([0., 0., 0.]),
 'LMTarsus3': array([0., 0., 0.]),
 'LMTarsus4': array([0., 0., 0.]),
 'LMTarsus5': array([0., 0., 

In [25]:
visual_offsets

{'A1A2': array([0., 0., 0.]),
 'A3': array([0., 0., 0.]),
 'A4': array([0., 0., 0.]),
 'A5': array([0., 0., 0.]),
 'A6': array([0., 0., 0.]),
 'Haustellum': array([0., 0., 0.]),
 'Head': array([0., 0., 0.]),
 'LAntenna': array([0., 0., 0.]),
 'LEye': array([0., 0., 0.]),
 'LFCoxa': array([ 0.    ,  0.    , -0.2125]),
 'LFFemur': array([ 0.   ,  0.   , -0.353]),
 'LFTarsus1': array([ 0.    ,  0.    , -0.3315]),
 'LFTibia': array([ 0.   ,  0.   , -0.259]),
 'LHCoxa': array([0., 0., 0.]),
 'LHFemur': array([0., 0., 0.]),
 'LHTarsus1': array([0., 0., 0.]),
 'LHTarsus2': array([0., 0., 0.]),
 'LHTarsus3': array([0., 0., 0.]),
 'LHTarsus4': array([0., 0., 0.]),
 'LHTarsus5': array([0., 0., 0.]),
 'LHTibia': array([0., 0., 0.]),
 'LHaltere': array([0., 0., 0.]),
 'LMCoxa': array([0., 0., 0.]),
 'LMFemur': array([0., 0., 0.]),
 'LMTarsus1': array([0., 0., 0.]),
 'LMTarsus2': array([0., 0., 0.]),
 'LMTarsus3': array([0., 0., 0.]),
 'LMTarsus4': array([0., 0., 0.]),
 'LMTarsus5': array([0., 0., 

In [26]:
body_pos["Rostrum"], body_pos["Head"], body_pos["Haustellum"]

(array([9.12983611e-01, 6.79564209e-04, 9.98466043e-01]),
 array([ 4.97922767e-01, -6.53750276e-07,  1.21077115e+00]),
 array([9.13176627e-01, 2.34613125e-04, 6.27014553e-01]))